In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

In [2]:
from sophius.db import *
from peewee import *
import pandas as pd

query = (Runs
         .select(Runs.id.alias('run_id'), Runs, Models, Devices.name.alias('dev_name'))
         .join(Models)
         .switch(Runs)
         .join(Devices)
         .dicts())

cols = ['run_id', 'hash', 'flops', 'params', 'val_acc', 'train_acc', 'time', 'dev_name']
df = pd.DataFrame(query)[cols]
df.time = df.time.round()

# df = pd.DataFrame(query)

df[-5:]

,run_id,hash,flops,params,val_acc,train_acc,time,dev_name
8537,8538,00808091200000000044a000400000000022b00001000000,4142496,14506,0.455512,0.465082,60.0,NVIDIA RTX A500 Laptop GPU
8538,8539,00880821200000000029480001000000,371416,82034,0.415451,0.419423,2.0,NVIDIA RTX A500 Laptop GPU
8539,8540,008020462000000000485000400000000024a80001000000,3658560,30730,0.370660,0.376150,3.0,NVIDIA RTX A500 Laptop GPU
8540,8541,00810821080800000022b0000010000001000000,1674432,1546,0.223112,0.217990,14.0,NVIDIA RTX A500 Laptop GPU
8541,8542,008200a2100400000044a000400000000023480001000000,7866816,107242,0.689143,0.778115,32.0,Quadro RTX 6000


In [3]:
from sophius.encode import Encoder

encoder = Encoder()
encoder.hash2model('00810112200000000044a000400000000080242208040000400000000010000001000000', (3, 32, 32), 10)

C:\Users\alexander.korolyov\miniforge3\envs\sophius\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Conv2d       (64, 16, 16)   (5, 5)   (2, 2)  
ReLU         (64, 16, 16)  
MaxPool2d    (64, 8, 8)     (3, 3)   (2, 2)  
BatchNorm2d  (64, 8, 8)    
Conv2d       (192, 8, 8)    (3, 3)   (1, 1)  
PReLU        (192, 8, 8)   
BatchNorm2d  (192, 8, 8)   
GlobalAvgPool2d (192, 1, 1)   
Flatten      192           
Linear       10            